## Drone Detection

In [ ]:
# essential imports
import os
import csv
import cv2
import json
import numpy as np

# plotting and display
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, clear_output

# import YOLO model from ultralytics package
from ultralytics import YOLO

# custom utilities
from utils import * #set_device

# reflect changes in src code immediately without restarting kernel
%load_ext autoreload
%autoreload 2

In [ ]:
# set the device depending on available GPU
device = set_device()

In [ ]:
# load pretrained YOLOv8 model
model = YOLO("yolov8m.pt")  # n for nano, s for small, m for medium, l for large, x for extra large

#### Try on drone video

In [ ]:
# all datasets path
data_path = './data/drone-tracking-datasets/'

# select dataset and camera
dataset_num = 3
cam_num = 0
video_path = os.path.join(data_path, f'dataset{dataset_num}/cam{cam_num}.mp4')

# get sample frame and store width and height
cap = cv2.VideoCapture(video_path)
ret, sample_frame = cap.read()
cap.release()
sample_frame_rgb = cv2.cvtColor(sample_frame, cv2.COLOR_BGR2RGB)
frame_height, frame_width, _ = sample_frame.shape
print(f"Frame size: {frame_width} x {frame_height}")
plt.imshow(sample_frame_rgb)

# select tracker
tracker = "sort.yaml"  # or 'bytetrack.yaml', 'strongsort.yaml'

In [ ]:
# run detection on video
results = model.predict(
    source=video_path, 
    device=device,   # set the device
    tracker=tracker,
    show=True, # display live output in external window (do not return generator)
    save=True,  # save output video with bounding boxes to runs/detect/
    #stream=True  # return a generator that yields results for each frame, doesn't store whole video in memory
)

In [ ]:
# test tracking

# save trajectory to CSV
csv_file = "trajectory_1.csv"

trajectory = []  # list to store (frame_id, x_center, y_center)

for frame_id, result in enumerate(results, start=1):
    if len(result.boxes) > 0:
        # Take first detection (assuming single drone)
        box = result.boxes[0].xywh[0]  # [x_center, y_center, width, height] in pixels
        x_center, y_center = box[0].item(), box[1].item()
    else:
        # If no detection, log NaN or skip
        x_center, y_center = float('nan'), float('nan')

    trajectory.append([frame_id, x_center, y_center])

# save to CSV
with open(csv_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["frame_id", "x_center", "y_center"])
    writer.writerows(trajectory)

print(f"Trajectory saved to {csv_file}")

In [ ]:
# Extract x, y values in pixels, skipping NaNs
x_vals = np.array([x for _, x, y in trajectory])
y_vals = np.array([y for _, x, y in trajectory])

# Threshold in pixels
distance_threshold = 50

# Prepare lists of segments
segments_x = []
segments_y = []

current_seg_x = []
current_seg_y = []

for i in range(len(x_vals)):
    x, y = x_vals[i], y_vals[i]
    
    if np.isnan(x) or np.isnan(y):
        # Missing detection → start new segment
        if current_seg_x:
            segments_x.append(current_seg_x)
            segments_y.append(current_seg_y)
            current_seg_x, current_seg_y = [], []
        continue
    
    if current_seg_x:
        # Compute distance to previous point
        dx = x - current_seg_x[-1]
        dy = y - current_seg_y[-1]
        dist = np.sqrt(dx**2 + dy**2)
        if dist > distance_threshold:
            # Jump too far → start new segment
            segments_x.append(current_seg_x)
            segments_y.append(current_seg_y)
            current_seg_x, current_seg_y = [], []

    # Add point to current segment
    current_seg_x.append(x)
    current_seg_y.append(y)

# Add last segment
if current_seg_x:
    segments_x.append(current_seg_x)
    segments_y.append(current_seg_y)

# Plot
plt.figure(figsize=(12, 8))
plt.imshow(sample_frame_rgb)
for seg_x, seg_y in zip(segments_x, segments_y):
    plt.plot(seg_x, seg_y, marker='o', color='red', linewidth=2, markersize=1)

#plt.title("Drone Trajectory Over Sample Frame (with breaks)")
plt.axis("off")
plt.show()


In [ ]:
# Prepare X and Y lists with NaNs for missing detections
x_vals = np.array([x if not np.isnan(x) else np.nan for _, x, y in trajectory])
y_vals = np.array([y if not np.isnan(y) else np.nan for _, x, y in trajectory])

plt.figure(figsize=(12, 8))
plt.imshow(sample_frame_rgb)

# Plot segments separately to break lines at NaNs
# Using np.ma (masked arrays) to ignore NaNs in line segments
plt.plot(np.ma.masked_invalid(x_vals), np.ma.masked_invalid(y_vals),
         marker='o', color='red', linewidth=2, markersize=2)

plt.title("Drone Trajectory Over Sample Frame")
plt.axis("off")
plt.show()

In [ ]:
# Filter out frames where drone was not detected
x_vals = [x for _, x, y in trajectory if not (x != x)]  # ignore NaNs
y_vals = [y for _, x, y in trajectory if not (y != y)]

# plot trajectory over sample frame
plt.figure(figsize=(12, 8))
plt.imshow(sample_frame_rgb)
plt.plot(x_vals, y_vals, marker='o', color='red', linewidth=2, markersize=4)
plt.title("Drone Trajectory Over Sample Frame")
plt.axis("off")
plt.show()

In [ ]:
# Counters
detected_count = 0
missed_count = 0

for result in results:
    if len(result.boxes) > 0:  # any detections in this frame?
        detected_count += 1
    else:
        missed_count += 1

print(f"Detected in {detected_count} frames")
print(f"Missed in {missed_count} frames")

In [ ]:
# for the detections in camera 0 in the first dataset
# there are 5334 frames total
# of these there are 2416 frames that have no detections
# so there are 2918 frames with detections
# 55% detection rate

# for the nano model
# Detected in 1606 frames
# Missed in 3728 frames
# 30% detection rate

# for the small model
# Detected in 2357 frames
# Missed in 2977 frames
# 44% detection rate, much better!

# for the medium model
# Detected in 2704 frames
# Missed in 2630 frames
# 50.7% detection rate, even better!


In [ ]:
count_zero = 0
total_frames = 0
file_name = os.path.join(data_path, 'dataset1/detections/cam0.txt')  

# Replace 'data.txt' with your file path
with open(file_name, "r") as f:
    for line in f:
        total_frames += 1
        # Skip empty lines
        if not line.strip():
            continue
        
        # Split line into columns
        cols = line.strip().split()
        
        # Convert 2nd and 3rd columns to float
        col2, col3 = float(cols[1]), float(cols[2])
        
        # Check if both are 0
        if col2 == 0.0 and col3 == 0.0:
            count_zero += 1

print(f"Number of rows where 2nd and 3rd columns are 0: {count_zero}")
print(f"Total number of frames: {total_frames}")

In [ ]:
# display results frame by frame inline, by iterating through the generator
# otherwise can open in external window with show=True above

for result in results:
    annotated_frame = result.plot()
    annotated_frame = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)

    plt.imshow(annotated_frame)
    plt.axis("off")
    clear_output(wait=True)  # clear previous cell output
    display(plt.gcf())

In [ ]:
# Grab first frame only
first_result = next(iter(results))

# Plot with bounding boxes
annotated_frame = first_result.plot()  # numpy array, BGR format
annotated_frame = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)

# Show with matplotlib
plt.figure(figsize=(10, 6))
plt.imshow(annotated_frame)
plt.axis("off")
plt.title("First Frame with YOLO Detections")
plt.show()

In [ ]:
# put training code into YOLO format

# <class_id> <x_center_norm> <y_center_norm> <width_norm> <height_norm>

# need to decide bbox size or compute as we go from the results?

In [ ]:
# Parameters
bbox_width = 20   # in pixels
bbox_height = 20  # in pixels
image_width = 1280  # replace with actual image width
image_height = 720  # replace with actual image height
class_id = 0

input_folder = "detections"  # folder with txt files
output_folder = "labels"     # YOLO format labels

os.makedirs(output_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    if not file_name.endswith(".txt"):
        continue

    with open(os.path.join(input_folder, file_name), "r") as f_in, \
         open(os.path.join(output_folder, file_name), "w") as f_out:

        for line in f_in:
            frame_id, x, y = line.strip().split()
            x = float(x)
            y = float(y)

            # Normalize
            x_norm = x / image_width
            y_norm = y / image_height
            w_norm = bbox_width / image_width
            h_norm = bbox_height / image_height

            f_out.write(f"{class_id} {x_norm:.6f} {y_norm:.6f} {w_norm:.6f} {h_norm:.6f}\n")


#### Plot the trajectory

In [ ]:
# load text file
data_path = './data/drone-tracking-datasets/'
trajectory_path = os.path.join(data_path, 'dataset3/trajectory/rtk.txt')

In [ ]:
plot_trajectory_static(trajectory_path)

In [ ]:
plot_trajectory_static_colored(trajectory_path)

In [ ]:
plot_trajectory_interactive(trajectory_path)

In [ ]:
import numpy as np


# Load trajectory
#trajectory = np.loadtxt("trajectory.txt")
x, y, z = trajectory[:,0], trajectory[:,1], trajectory[:,2]

# Create color scale based on time
t = np.linspace(0, 1, len(x))

fig = go.Figure(data=[go.Scatter3d(
    x=x, y=y, z=z,
    mode='markers+lines',
    marker=dict(
        size=5,
        color=t,
        colorscale='Viridis',
        showscale=True
    ),
    line=dict(
        color='gray',
        width=2
    )
)])

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title="Interactive 3D Drone Trajectory"
)

fig.show()


#### Triangulization

In [ ]:
# first i need to pick a reference camera for synchronization
# then i can align the frames from other cameras (maybe start with one)

# then when i have aligned frames that detect the xy coords of the drone at the same time
# i can triangulate the 3d position
# use the intrinsics (K and distortion params), and known camera positions to do this 

In [ ]:
# run the detection on all videos and save to file
data_path = './data/drone-tracking-datasets/'
dataset_num = 3
num_cams = 6

# select tracker
tracker = "sort.yaml"  # or 'bytetrack.yaml', 'strongsort.yaml'

for cam_num in range(num_cams):
    video_path = os.path.join(data_path, f'dataset{dataset_num}/cam{cam_num}.mp4')
    csv_file = os.path.join('dataset3_result_detections', f'detections_cam{cam_num}.txt')

    results = model.predict(source=video_path, tracker=tracker, save=False, stream=True)

    trajectory = []  # list to store (frame_id, x_center, y_center)

    for frame_id, result in enumerate(results, start=1):
        if len(result.boxes) > 0:
            # Take first detection (assuming single drone)
            box = result.boxes[0].xywh[0]  # [x_center, y_center, width, height] in pixels
            x_center, y_center = box[0].item(), box[1].item()
        else:
            # If no detection, log 0
            x_center, y_center = 0.0, 0.0

        trajectory.append([frame_id, x_center, y_center])

    # save to CSV
    with open(csv_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["frame_id", "x_center", "y_center"])
        writer.writerows(trajectory)

    print(f"Trajectory saved to {csv_file}")

In [ ]:
# If CSV has header
data = np.genfromtxt("trajectory.csv", delimiter=",", skip_header=1)

print(len(data))
print(data[35,1])

In [ ]:
# info for dataset 3 and cams

'''
cam0 - gopro3
cam1 - mate7
cam2 - mate10
cam3 - sony5n_1440x1080
cam4 - sony5100
cam5 - sonyG
'''

# locations
'''
44.535 11.56253333 -1.1467
4.3962 -54.27346667 4.0965
-42.5242 -21.00086667 -1.7639
40.1912 44.79053333 -0.9379
-34.8458 -44.00906667 1.4179
-11.7524 62.93033333 -1.6659
'''

In [ ]:
# sync params for time shift and scale

# Time scale (alpha) matrix
alpha = np.array([
    [1.0000, 0.5005, 0.4960, 0.4171, 0.5000, 0.8341],
    [1.9982, 1.0000, 0.9910, 0.8333, 0.9990, 1.6667],
    [2.0163, 1.0091, 1.0000, 0.8409, 1.0081, 1.6819],
    [2.3978, 1.2000, 1.1892, 1.0000, 1.1988, 2.0000],
    [2.0001, 1.0010, 0.9919, 0.8342, 1.0000, 1.6683],
    [1.1989, 0.6000, 0.5946, 0.5000, 0.5994, 1.0000],
])

# Time shift (beta) matrix
beta = np.array([
    [0.00,     1013.95,  546.98,  251.16,  961.02,  137.51],
    [-2026.04,    0.00, -457.83, -593.82,  -51.96, -1552.47],
    [-1102.90,  461.99,    0.00, -208.81,  409.59,  -782.45],
    [ -602.21,  712.57,  248.32,    0.00,  659.93,  -364.81],
    [-1922.12,   52.01, -406.29, -551.00,    0.00, -1465.78],
    [ -164.85,  931.45,  465.22,  182.40,  878.60,     0.00],
])

# decide ref cam
ref_cam_num = 0

# isolate relevant rows for both matrics
alpha_ref = alpha[ref_cam_num, :]
beta_ref = beta[ref_cam_num, :]

print(alpha_ref)
print(beta_ref) 

In [ ]:
# get the intrinsics for the cameras from json files

camera = 'gopro3'
json_file = os.path.join(data_path, f'calibration/{camera}/{camera}.json')

# Load JSON
with open(json_file, "r") as f:
    cam_info = json.load(f)

# Access attributes
K = cam_info["K-matrix"]
dist = cam_info["distCoeff"]
fps = cam_info["fps"]
res = cam_info["resolution"]

print("K matrix:", K)
print("Distortion coeffs:", dist)
print("FPS:", fps)
print("Resolution:", res)

In [ ]:
# If CSV has header
data_1 = np.genfromtxt("trajectory.csv", delimiter=",", skip_header=1)
data_2 = np.genfromtxt("trajectory_1.csv", delimiter=",", skip_header=1)

# chose second camera
cam_num_2 = 1
# get sync params
alpha_2 = alpha_ref[cam_num_2]
beta_2 = beta_ref[cam_num_2]


# iterate through frames of reference camera
for frame_id in range(len(data)):

    # use sync info to get corresponding frame id from other camera
    j = alpha_2 * frame_id + beta_2
    if j < 0 or j >= len(data_2):
        continue  # skip if out of bounds

    # get x, y coords from both cameras
    x1, y1 = data_1[frame_id, 1], data_1[frame_id, 2]
    x2, y2 = data_2[int(j), 1], data_2[int(j), 2]

    # undistort points using cv2 method
    ray1 = cv2.undistortPoints(np.array([[[x1, y1]]], dtype=np.float32), K1, dist1, P=K1)
    ray2 = cv2.undistortPoints(np.array([[[x2, y2]]], dtype=np.float32), K2, dist2, P=K2)

    # transform rays to world coordinates
    

    # triangulate 3d point from rays


In [ ]:
# might be easier to run the detection on both videos and save to file, 
# then just get the frame id and the detection info directly